In [76]:
# 패키지 설치
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
log_data = pd.read_csv('/content/drive/MyDrive/dataset/log_data.csv')

In [79]:
relog_data = pd.read_csv('/content/drive/MyDrive/dataset/relog.csv')

In [80]:
log_data=log_data.sort_values(by=["user_id","timestamp","event"], ascending=[True,True,True])

In [81]:
log_data.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
11709372,1,GetCreditInfo,2022-05-03 14:52:28,android,464,2022-05-03
11709374,1,GetCreditInfo,2022-05-03 14:52:35,android,464,2022-05-03
2451693,1,Login,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
2451691,1,UseLoanManage,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
7071607,1,GetCreditInfo,2022-06-16 23:58:42,android,464,2022-06-16


In [82]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17843993 entries, 11709372 to 8248082
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   user_id         int64 
 1   event           object
 2   timestamp       object
 3   mp_os           object
 4   mp_app_version  object
 5   date_cd         object
dtypes: int64(1), object(5)
memory usage: 953.0+ MB


In [83]:
log_data.isnull().sum()

user_id                0
event                  0
timestamp              0
mp_os                980
mp_app_version    660597
date_cd                0
dtype: int64

1. 결측치 처리

In [84]:
log_data['mp_os'].value_counts()

Android    12331688
iOS         3312862
android     2195434
iPadOS         3029
Name: mp_os, dtype: int64

- 안드로이드 os 무슨차이가 있는지 확인

In [85]:
log_data[log_data['mp_os']=='Android']['event'].value_counts()

OpenApp                    2897107
Login                      2463755
EndLoanApply               2047047
StartLoanApply             1436048
ViewLoanApplyIntro         1369981
UseLoanManage              1186306
CompleteIDCertification     910773
SignUp                       20191
UsePrepayCalc                  269
UseDSRCalc                     211
Name: event, dtype: int64

In [86]:
log_data[log_data['mp_os']=='android']['event'].value_counts()

GetCreditInfo    2195434
Name: event, dtype: int64

- ios는 무슨차이 있는지 확인

In [87]:
log_data[log_data['mp_os']=='iOS']['event'].value_counts()

EndLoanApply               667585
OpenApp                    563415
GetCreditInfo              465583
StartLoanApply             457487
ViewLoanApplyIntro         434381
UseLoanManage              371868
CompleteIDCertification    326504
SignUp                      14497
UsePrepayCalc                7091
UseDSRCalc                   4451
Name: event, dtype: int64

In [88]:
log_data[log_data['mp_os']=='iPadOS']['event'].value_counts()

UseLoanManage              732
EndLoanApply               621
CompleteIDCertification    500
StartLoanApply             379
ViewLoanApplyIntro         350
OpenApp                    240
SignUp                     204
UseDSRCalc                   3
Name: event, dtype: int64

android os는 getcreditinfo행동밖에 없고, Android os는 getcreditinfo만 없으므로 합쳐주는게 맞다고 판단 android -> Android로 변환

In [89]:
log_data.loc[list(log_data[log_data['mp_os']=='android'].index),'mp_os'] = 'Android'

In [90]:
log_data['mp_os'].value_counts()

Android    14527122
iOS         3312862
iPadOS         3029
Name: mp_os, dtype: int64

In [91]:
log_null = log_data[log_data['mp_os'].isnull()]
log_null

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
10641598,2109,GetCreditInfo,2022-03-08 12:38:52,NaN,NaN,2022-03-08
11276759,2109,GetCreditInfo,2022-03-22 21:49:01,NaN,NaN,2022-03-22
11083776,2109,GetCreditInfo,2022-03-23 21:14:54,NaN,NaN,2022-03-23
14021782,2231,GetCreditInfo,2022-05-24 20:20:52,NaN,NaN,2022-05-24
13995185,2231,GetCreditInfo,2022-05-24 22:50:24,NaN,NaN,2022-05-24
...,...,...,...,...,...,...
9406160,852313,GetCreditInfo,2022-03-18 12:44:15,NaN,NaN,2022-03-18
11225134,852313,GetCreditInfo,2022-06-08 21:04:33,NaN,NaN,2022-06-08
10921708,852313,GetCreditInfo,2022-06-14 20:20:31,NaN,NaN,2022-06-14
11111001,871149,GetCreditInfo,2022-03-23 16:01:41,NaN,NaN,2022-03-23


mp_os가 결측값이면 mp_app_version도 결측값임을 확인

- 그냥 mp_os랑 mp_app_version 컬럼 무시하기?!?!?!?!

# log_data에 파생변수 넣기

### 파생변수 1 : 각 유저가 하루에 행동을 얼마나 했는지

In [92]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
grouped

user_id  date_cd   
1        2022-05-03     2
         2022-06-16     3
7        2022-05-22     1
9        2022-05-21     3
11       2022-03-24    12
                       ..
879693   2022-06-29     1
879694   2022-03-31     6
879695   2022-05-27     4
879696   2022-03-14    11
879698   2022-05-24     3
Name: user_id, Length: 2728153, dtype: int64

In [93]:
df_grouped = pd.DataFrame(grouped)
df_grouped

user_id
user_id date_cd            
1       2022-05-03        2
        2022-06-16        3
7       2022-05-22        1
9       2022-05-21        3
11      2022-03-24       12
...                     ...
879693  2022-06-29        1
879694  2022-03-31        6
879695  2022-05-27        4
879696  2022-03-14       11
879698  2022-05-24        3

[2728153 rows x 1 columns]

In [94]:
df_grouped.rename(columns = {'user_id' : 'user_id_cnt'}, inplace = True)
df_grouped

user_id_cnt
user_id date_cd                
1       2022-05-03            2
        2022-06-16            3
7       2022-05-22            1
9       2022-05-21            3
11      2022-03-24           12
...                         ...
879693  2022-06-29            1
879694  2022-03-31            6
879695  2022-05-27            4
879696  2022-03-14           11
879698  2022-05-24            3

[2728153 rows x 1 columns]

In [95]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new

,date_cd,user_id_cnt
user_id,,
1,2022-05-03,2
1,2022-06-16,3
7,2022-05-22,1
9,2022-05-21,3
11,2022-03-24,12
...,...,...
879693,2022-06-29,1
879694,2022-03-31,6
879695,2022-05-27,4


In [96]:
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new

,user_id,date_cd,user_id_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12
...,...,...,...
2728148,879693,2022-06-29,1
2728149,879694,2022-03-31,6
2728150,879695,2022-05-27,4
2728151,879696,2022-03-14,11


### 파생변수 2 : 유저가 했던 행동을 총합해서 하나의 컬럼으로 추출

In [97]:
sample=pd.DataFrame(df_grouped_new['user_id_cnt'].groupby(df_grouped_new['user_id']).sum())
sample

,user_id_cnt
user_id,
1,5
7,1
9,3
11,43
12,120
...,...
879693,61
879694,6
879695,4


In [98]:
relog_data = pd.merge(relog_data , sample,
                 how = 'left',
                 on = 'user_id')
relog_data

,Unnamed: 0,user_id,timestamp,event,user_id_cnt
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11,5
1,10428909,7,2022-05-22 16:39:49,11,1
2,9627339,9,2022-05-21 23:37:58,11-11-11,3
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120
...,...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7,4
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11


In [99]:
relog_data

,Unnamed: 0,user_id,timestamp,event,user_id_cnt
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11,5
1,10428909,7,2022-05-22 16:39:49,11,1
2,9627339,9,2022-05-21 23:37:58,11-11-11,3
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120
...,...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7,4
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11


### 파생변수3 : 유저가 기간동안 총 몇일 접속했는지

In [100]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
grouped

user_id  date_cd   
1        2022-05-03     2
         2022-06-16     3
7        2022-05-22     1
9        2022-05-21     3
11       2022-03-24    12
                       ..
879693   2022-06-29     1
879694   2022-03-31     6
879695   2022-05-27     4
879696   2022-03-14    11
879698   2022-05-24     3
Name: user_id, Length: 2728153, dtype: int64

In [101]:
df_grouped = pd.DataFrame(grouped)
df_grouped

user_id
user_id date_cd            
1       2022-05-03        2
        2022-06-16        3
7       2022-05-22        1
9       2022-05-21        3
11      2022-03-24       12
...                     ...
879693  2022-06-29        1
879694  2022-03-31        6
879695  2022-05-27        4
879696  2022-03-14       11
879698  2022-05-24        3

[2728153 rows x 1 columns]

In [102]:
df_grouped.rename(columns = {'user_id' : 'user_id_day_cnt'}, inplace = True)
df_grouped

user_id_day_cnt
user_id date_cd                    
1       2022-05-03                2
        2022-06-16                3
7       2022-05-22                1
9       2022-05-21                3
11      2022-03-24               12
...                             ...
879693  2022-06-29                1
879694  2022-03-31                6
879695  2022-05-27                4
879696  2022-03-14               11
879698  2022-05-24                3

[2728153 rows x 1 columns]

In [103]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new

,date_cd,user_id_day_cnt
user_id,,
1,2022-05-03,2
1,2022-06-16,3
7,2022-05-22,1
9,2022-05-21,3
11,2022-03-24,12
...,...,...
879693,2022-06-29,1
879694,2022-03-31,6
879695,2022-05-27,4


In [104]:
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new

,user_id,date_cd,user_id_day_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12
...,...,...,...
2728148,879693,2022-06-29,1
2728149,879694,2022-03-31,6
2728150,879695,2022-05-27,4
2728151,879696,2022-03-14,11


In [105]:
df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count()

user_id
1          2
7          1
9          1
11         4
12        14
          ..
879693     8
879694     1
879695     1
879696     1
879698     1
Name: user_id_day_cnt, Length: 584636, dtype: int64

In [106]:
sample1=pd.DataFrame(df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count())
sample1

,user_id_day_cnt
user_id,
1,2
7,1
9,1
11,4
12,14
...,...
879693,8
879694,1
879695,1


In [107]:
relog_data = pd.merge(relog_data , sample1,
                 how = 'left',
                 on = 'user_id')
relog_data

,Unnamed: 0,user_id,timestamp,event,user_id_cnt,user_id_day_cnt
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11,5,2
1,10428909,7,2022-05-22 16:39:49,11,1,1
2,9627339,9,2022-05-21 23:37:58,11-11-11,3,1
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43,4
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120,14
...,...,...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61,8
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6,1
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7,4,1
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11,1


### 파생변수 4: 한도 조회, 신용 정보 조회, 대출 관리 서비스 이용, 여윳돈 계산기 서비스 이용, DSR 계산기 서비스 이용 중 어떤 것을 먼저 이용한 사람인가에 따른 차이점

In [108]:
log_data_1 = log_data[log_data['event'] == 'EndLoanApply']
log_data_2 = log_data_1.append(log_data[log_data['event'] == 'UseLoanManage'])
log_data_3 = log_data_2.append(log_data[log_data['event'] == 'UsePrepayCalc'])
log_data_4 = log_data_3.append(log_data[log_data['event'] == 'UseDSRCalc'])
log_data_new = log_data_4.append(log_data[log_data['event'] == 'GetCreditInfo'])

In [109]:
group = log_data_new.groupby(['user_id', 'event'])['timestamp'].min()
group

user_id  event        
1        GetCreditInfo    2022-05-03 14:52:28
         UseLoanManage    2022-06-16 23:58:41
7        GetCreditInfo    2022-05-22 16:39:49
9        GetCreditInfo    2022-05-21 23:37:58
11       EndLoanApply     2022-03-24 10:59:46
                                 ...         
879694   EndLoanApply     2022-03-31 20:18:18
879695   EndLoanApply     2022-05-27 12:51:23
879696   EndLoanApply     2022-03-14 05:42:14
         GetCreditInfo    2022-03-14 05:35:43
         UseLoanManage    2022-03-14 05:35:41
Name: timestamp, Length: 1070648, dtype: object

In [110]:
df_group = pd.DataFrame(group)
df_group

timestamp
user_id event                             
1       GetCreditInfo  2022-05-03 14:52:28
        UseLoanManage  2022-06-16 23:58:41
7       GetCreditInfo  2022-05-22 16:39:49
9       GetCreditInfo  2022-05-21 23:37:58
11      EndLoanApply   2022-03-24 10:59:46
...                                    ...
879694  EndLoanApply   2022-03-31 20:18:18
879695  EndLoanApply   2022-05-27 12:51:23
879696  EndLoanApply   2022-03-14 05:42:14
        GetCreditInfo  2022-03-14 05:35:43
        UseLoanManage  2022-03-14 05:35:41

[1070648 rows x 1 columns]

In [111]:
df_group_new = df_group.reset_index()
df_group_new

,user_id,event,timestamp
0,1,GetCreditInfo,2022-05-03 14:52:28
1,1,UseLoanManage,2022-06-16 23:58:41
2,7,GetCreditInfo,2022-05-22 16:39:49
3,9,GetCreditInfo,2022-05-21 23:37:58
4,11,EndLoanApply,2022-03-24 10:59:46
...,...,...,...
1070643,879694,EndLoanApply,2022-03-31 20:18:18
1070644,879695,EndLoanApply,2022-05-27 12:51:23
1070645,879696,EndLoanApply,2022-03-14 05:42:14
1070646,879696,GetCreditInfo,2022-03-14 05:35:43


In [112]:
df_final = df_group_new.drop_duplicates(['user_id'], keep = 'first')
df_final

,user_id,event,timestamp
0,1,GetCreditInfo,2022-05-03 14:52:28
2,7,GetCreditInfo,2022-05-22 16:39:49
3,9,GetCreditInfo,2022-05-21 23:37:58
4,11,EndLoanApply,2022-03-24 10:59:46
8,12,EndLoanApply,2022-03-14 01:16:26
...,...,...,...
1070637,879692,EndLoanApply,2022-04-18 17:05:32
1070640,879693,EndLoanApply,2022-05-13 11:34:29
1070643,879694,EndLoanApply,2022-03-31 20:18:18
1070644,879695,EndLoanApply,2022-05-27 12:51:23


In [113]:
df_final.rename(columns = {'event':'first_event'},inplace=True)
df_final

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id,first_event,timestamp
0,1,GetCreditInfo,2022-05-03 14:52:28
2,7,GetCreditInfo,2022-05-22 16:39:49
3,9,GetCreditInfo,2022-05-21 23:37:58
4,11,EndLoanApply,2022-03-24 10:59:46
8,12,EndLoanApply,2022-03-14 01:16:26
...,...,...,...
1070637,879692,EndLoanApply,2022-04-18 17:05:32
1070640,879693,EndLoanApply,2022-05-13 11:34:29
1070643,879694,EndLoanApply,2022-03-31 20:18:18
1070644,879695,EndLoanApply,2022-05-27 12:51:23


In [115]:
sample3 = df_final[['user_id','first_event']]

In [116]:
relog_data = pd.merge(relog_data , sample3,
                 how = 'left',
                 on = 'user_id')
relog_data

,Unnamed: 0,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11,5,2,GetCreditInfo
1,10428909,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo
2,9627339,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43,4,EndLoanApply
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120,14,EndLoanApply
...,...,...,...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61,8,EndLoanApply
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6,1,EndLoanApply
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7,4,1,EndLoanApply
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11,1,EndLoanApply


### first_event가 널값인 경우는 한도 조회, 신용 정보 조회, 대출 관리 서비스 이용, 여윳돈 계산기 서비스 이용, DSR 계산기 서비스 이용 중 어떠한 것도 이용한 사람이 아니므로 others 부여(73932개의 유저아이디에 해당함 584636개중에서..)

In [117]:
relog_data['first_event'].isnull().sum()

73932

In [118]:
relog_data['first_event']

0         GetCreditInfo
1         GetCreditInfo
2         GetCreditInfo
3          EndLoanApply
4          EndLoanApply
              ...      
584631     EndLoanApply
584632     EndLoanApply
584633     EndLoanApply
584634     EndLoanApply
584635              NaN
Name: first_event, Length: 584636, dtype: object

In [126]:
relog_data = relog_data.fillna('others')

In [127]:
relog_data

,Unnamed: 0,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11,5,2,GetCreditInfo
1,10428909,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo
2,9627339,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43,4,EndLoanApply
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120,14,EndLoanApply
...,...,...,...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61,8,EndLoanApply
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6,1,EndLoanApply
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7,4,1,EndLoanApply
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11,1,EndLoanApply


In [128]:
relog_data.isnull().sum()

Unnamed: 0         0
user_id            0
timestamp          0
event              0
user_id_cnt        0
user_id_day_cnt    0
first_event        0
dtype: int64

### 필요없다고 생각되는 mp_os, mp_app_version 컬럼삭제

In [131]:
relog_data_final = relog_data.drop(['Unnamed: 0'],axis=1)
relog_data_final

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event
0,1,2022-05-03 14:52:28,11-11-2-8-11,5,2,GetCreditInfo
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo
3,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...,43,4,EndLoanApply
4,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...,120,14,EndLoanApply
...,...,...,...,...,...,...
584631,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...,61,8,EndLoanApply
584632,879694,2022-03-31 20:07:23,6-5-4-7-7-7,6,1,EndLoanApply
584633,879695,2022-05-27 12:48:32,6-5-4-7,4,1,EndLoanApply
584634,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7,11,1,EndLoanApply


In [136]:
relog_data_final.to_csv('/content/drive/MyDrive/dataset/relog_1.csv')

mp_os가 결측값인 유저중에 같은 유저가 mp_os 결측값이 아닌경우가 있는지 확인

In [ ]:
# 유저 성별이 다른 신청서에는 있지만 결측치가 있는 유저들 처리
sex_null_list = set(user_spec[(user_spec['유저_번호'].isin(set(user_spec[user_spec['유저_성별'].isnull()]['유저_번호'])))&(user_spec['유저_성별'].notnull())]['유저_번호'])
for i in sex_null_list:
    user_spec.loc[(user_spec['유저_번호']==i)&(user_spec['유저_생년월일'].isnull()),'유저_생년월일'] = user_spec[user_spec['유저_번호'] ==i]['유저_생년월일'].dropna().unique().item()
    user_spec.loc[(user_spec['유저_번호']==i)&(user_spec['유저_성별'].isnull()),'유저_성별'] = user_spec[user_spec

TypeError: ignored

In [ ]:
# 유저 성별이 다른 신청서에는 있지만 결측치가 있는 유저들 처리
mpos_null_list = set(log_data[(log_data['user_id'].isin(set(log_data[log_data['mp_os'].isnull()]['user_id'])))&(log_data['mp_os'].notnull())]['user_id'])

In [ ]:
mpos_null_list

{2109,
 2231,
 12238,
 13052,
 15279,
 21911,
 29338,
 39958,
 45865,
 92115,
 114258,
 140219,
 147815,
 173439,
 174650,
 186748,
 197415,
 207017,
 212026,
 220249,
 238063,
 243710,
 276883,
 307176,
 309918,
 317412,
 329617,
 352754,
 371426,
 371601,
 387116,
 390035,
 400047,
 405202,
 406413,
 408255,
 435937,
 445361,
 494336,
 510091,
 518658,
 529274,
 573815,
 583660,
 599742,
 601422,
 615687,
 642533,
 643345,
 653151,
 660962,
 666384,
 669844,
 678379,
 692250,
 695381,
 720571,
 725353,
 784112,
 797268,
 810429,
 846900,
 851105,
 852313,
 871149}

### mp_os/mp_app_version은 의미가 없는 컬럼이라고 판단. 컬럼 삭제 진행

# 세션잡아버리기 OpenApp기준

대출한도조회를 해야 대출을 할수 있으므로 한도조회를 기준으로 세션잡기

In [ ]:
relog_data

,Unnamed: 0,user_id,timestamp,event
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11
1,10428909,7,2022-05-22 16:39:49,11
2,9627339,9,2022-05-21 23:37:58,11-11-11
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...
...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7
